# Example: using OpenMM to run a simulation with a NequIP ML potential

You can run this example directly in your browser: [![Open On Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sef43/openmm-ml/blob/nequip/examples/nequip/run_nequip.ipynb)

## Install Conda

[Conda](https://docs.conda.io/) is a package and environment manager. On Google Colab, Conda is installed with [conda-colab](https://github.com/jaimergp/condacolab). On your computer, you should follow these [installation instructions](https://docs.conda.io/projects/conda/en/latest/user-guide/install/index.html).

⚠️ Do not use conda-colab on your computer!

In [ ]:
!pip install -q condacolab
import condacolab

condacolab.install_mambaforge()  # use mamba on colab because it is faster than conda

## Install software

First install everything we can from [conda-forge](https://conda-forge.org/).
Then use pip.

⚠️ The installation might take up to 10 min!

In [ ]:
!mambda install -c conda-forge openmm-ml nnpops
!pip install git+https://github.com/mir-group/nequip@develop

## Get the files we need to run the example

In [ ]:
!wget https://raw.githubusercontent.com/openmm/openmm-ml/main/examples/nequip/toluene.pdb
!wget https://raw.githubusercontent.com/openmm/openmm-ml/main/examples/nequip/toluene-deployed.pth

## Run simulation

In [2]:
from sys import stdout

import openmm
import openmm.app as app
import openmm.unit as unit

from openmmml import MLPotential

"""
Uses a deployed trained NequIP model from the basic example on https://github.com/mir-group/nequip
>>> nequip-train configs/example.yaml
>>> nequip-deploy build --train-dir path/to/training/session/ toluene-deployed.pth
"""

# Load toluene structure
pdb = app.PDBFile("toluene.pdb")

# Create a System with NequIP MLP

# Need to specify the unit conversion factors from the NequIP model units to OpenMM units.
# Distance: model is in Angstrom, OpenMM is in nanometers
A_to_nm = 0.1
# Energy: model is in kcal/mol, OpenMM is in kJ/mol
kcal_to_kJ_per_mol = 4.184

potential = MLPotential(
    "nequip",
    modelPath="toluene-deployed.pth",
    lengthScale=A_to_nm,
    energyScale=kcal_to_kJ_per_mol,
)

system = potential.createSystem(pdb.topology)

# Run langevin dynamics at 300K for 1000 steps
integrator = openmm.LangevinIntegrator(
    300 * unit.kelvin, 1.0 / unit.picoseconds, 1.0 * unit.femtosecond
)
simulation = app.Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(pdb.positions)
simulation.reporters.append(app.DCDReporter("output.dcd", 100))
simulation.reporters.append(
    app.StateDataReporter(
        stdout, 100, step=True, potentialEnergy=True, temperature=True, speed=True
    )
)

# Minimize the energy
simulation.minimizeEnergy()

# Set the velocities to 300K and run 1000 steps
simulation.context.setVelocitiesToTemperature(300*unit.kelvin)
simulation.step(1000)

#"Step","Potential Energy (kJ/mole)","Temperature (K)"
100,-710452.5,220.47627134902876
200,-710464.5625,204.22321067994645
300,-710484.25,312.68608178697696
400,-710462.375,250.56526717177863
500,-710464.25,289.43386954186155
600,-710448.9375,219.63591412548993
700,-710446.0,261.7770563640068
800,-710466.75,454.97388216259844
900,-710454.0,401.38716825310536
1000,-710460.5,429.09933310001867
-710513.4375 kJ/mol
